<a href="https://colab.research.google.com/github/MayaHayat/EconAlgo_Ex6Q4/blob/main/EconAlgo_Ex6_Question4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import networkx as nx
import cvxpy
import matplotlib.pyplot as plt
import numpy as np
import math
import doctest

In [2]:
def get_player_room(node1, node2):
  id1, id2 = int(node1.split('_')[-1]), int(node2.split('_')[-1])
  if 'player' in node1:
    return id1, id2
  elif 'player' in node2:
    return id2, id1
  else:
    print("Invalid node format:", node1, node2)
    exit(1)


In [13]:
def find_rent_with_nonnegative_prices(valuations: list[list[float]], price):
    G = nx.Graph()  # Create an undirected graph

    num_players = len(valuations)
    num_rooms = len(valuations[0])

    # ------------------------------- Part 1: match rooms - players -------------------------------

    # Add players and rooms as nodes
    for i in range(num_players):
        player_name = f"player_{i}"
        #player_name = i
        G.add_node(player_name, bipartite=0)

    for j in range(num_rooms):
        room_name = f"room_{j}"
        #room_name = j
        G.add_node(room_name, bipartite=1)

    # Add edges with weights and named nodes
    for i in range(num_players):
        for j in range(num_rooms):
            weight = valuations[i][j]
            if weight > 0:
                G.add_edge(f"player_{i}", f"room_{j}", weight=weight)

    max_weight_matching = nx.max_weight_matching(G)
    print("Max matching", max_weight_matching)

    # Create a dict to store player-room assignments
    matches = {}
    for node1, node2 in max_weight_matching:
      player_id, room_id = get_player_room(node1, node2)
      matches[player_id] = room_id
    print(matches)

    # ------------------------------- Part 2 : set pricing -------------------------------
    variables = [cvxpy.Variable() for _ in range(num_rooms)]
    fixed_constraints = [sum(variables) == price]
    for player_id in range(num_players):
      matched_room_id = matches[player_id]
      for room_id in range(num_rooms):
        if matched_room_id == room_id:
          continue
        constraint = (valuations[player_id][matched_room_id] - variables[matched_room_id]) \
                    >= (valuations[player_id][room_id] - variables[room_id])

        matched_room_id = matches[player_id]
        fixed_constraints.append(constraint)

    prob = cvxpy.Problem(cvxpy.Maximize(0), constraints=fixed_constraints)
    prob.solve()
    print("Status:", prob.status)
    print("Value:", prob.value)
    list = [1, 2, 3]

    print('rents: ' + ', '.join(['room_{}={:.3f}'.format(room_id, variable.value) for room_id, variable in enumerate(variables)]))
    return G

In [14]:
G = find_rent_with_nonnegative_prices( [[20,30,40],[40,30,20],[30,30,30]], 90)


Max matching {('room_2', 'player_0'), ('room_0', 'player_1'), ('player_2', 'room_1')}
{0: 2, 1: 0, 2: 1}
Status: optimal
Value: 0.0
rents: room_0=31.667, room_1=26.667, room_2=31.667


In [15]:
G = find_rent_with_nonnegative_prices( [[350,250,350],[600,400,400],[400,200,250]], 1000)


Max matching {('room_1', 'player_2'), ('player_0', 'room_2'), ('room_0', 'player_1')}
{2: 1, 0: 2, 1: 0}
Status: optimal
Value: 0.0
rents: room_0=439.085, room_1=239.085, room_2=321.831
